In [3]:
import pathlib

import numpy as np
import stim

from lomatching.greedy_algorithm import get_ops, get_time_hypergraph, get_track_ordering, check_ordering

# Example of a 2q Clifford circuit

In [4]:
stim.Tableau.random(2).to_circuit(method="elimination")

stim.Circuit('''
    H 0
    CX 0 1
    H 1
    CX 1 0
    S 1
    H 1
    S 1
    H 0 1
    S 0 0 1 1
    H 0 1
    S 1 1
''')

# Why improve the circuits?

```
array([['R', 'R'],
       ['S', 'I'],
       ['H', 'S'],
       ['CX0-1', 'CX0-1'],
       ['I', 'H'],
       ['CX1-0', 'CX1-0'],
       ['I', 'S'],
       ['I', 'H'],
       ['I', 'S'],
       ['I', 'H'], #
       ['I', 'S'], #
       ['I', 'S'], #
       ['S', 'H'], #
       ['S', 'I'],
       ['M', 'M']], dtype=object)
```

One can see that the highlighted line is to perform a `X` gate. This gate is "free" because it can be tracked in software. 

Therefore, we want to sample all 2-qubit Clifford circuits modulo Pauli gates. The Pauli gates only change the sign of the tableau generators (a Pauli string either commutes or anticommutes with a Pauli gate, the Pauli gate cannot change a e.g. Pauli X to a Pauli Y). 

This can be seen in the example below where we remove the signs in the tableau generators:

In [5]:
stim.Tableau.from_conjugated_generators(
    xs=[
        stim.PauliString("-YX"),
        stim.PauliString("-ZZ"),
    ],
    zs=[
        stim.PauliString("-_Z"),
        stim.PauliString("-XZ"),
    ],
).to_circuit(method="elimination")

stim.Circuit('''
    CX 1 0 0 1 1 0
    H 0 1
    CX 0 1
    H 1
    CX 1 0
    S 1
    H 1
    S 1 1
    H 1
    S 1 1
''')

In [6]:
stim.Tableau.from_conjugated_generators(
    xs=[
        stim.PauliString("+YX"),
        stim.PauliString("+ZZ"),
    ],
    zs=[
        stim.PauliString("+_Z"),
        stim.PauliString("+XZ"),
    ],
).to_circuit(method="elimination")

stim.Circuit('''
    CX 1 0 0 1 1 0
    H 0 1
    CX 0 1
    H 1
    CX 1 0
    S 1
''')

One can see that the circuit decomposition can be improved, as the line ``CX 1 0 0 1 1 0`` is basically doing a SWAP gate, which is free. This optimization of the SWAP gates is going to be performed later on. 

Now we will focus on generating all the 2q Clifford circuits module Paulis. There are 11520 2q Clifford circuits. 

In [7]:
len(list(stim.Tableau.iter_all(2)))

11520

There are 720 2q Clifford circuits modulo Paulis. This can be explained because if all the Paulis are commuted to the end of the circuit, then we have a Clifford circuit modulo Paulis + Paulis. For 2q, there are a total of 16 combinations of Paulis (`4*4, 4 = I,X,Y,Z`) from which we are only interested in 1, i.e. `I*I`. Therefore 11520/16=720. 

In [8]:
cliffords_mod_pauli = []
for t in stim.Tableau.iter_all(2):
    tn = t.to_numpy()
    tnew = stim.Tableau.from_numpy(x2x=tn[0], x2z=tn[1], z2x=tn[2], z2z=tn[3], x_signs=np.zeros(2, dtype=bool), z_signs=np.zeros(2, dtype=bool))
    if tnew not in cliffords_mod_pauli:
        cliffords_mod_pauli.append(tnew)

print(len(cliffords_mod_pauli))

720


The SWAP gates are also free if we assume all-to-all connectivity, which is reasonable if one wants to implement fold-transversal gates. Therefore, we can just simulate the 2q Clifford circuits modulo Paulis and module SWAPs. This reduces the size of the group to half because we can always make C1 and C1+SWAP. 

In [9]:
cliffords_mod_pauli_swappairs = []
swap_equiv_circuits_inds = {}
for i, tableau_1 in enumerate(cliffords_mod_pauli):
    a1, b1, c1, d1, v1, w1 = tableau_1.to_numpy()
    assert (v1 == 0).all() and (w1 == 0).all()
    prev_len = len(cliffords_mod_pauli_swappairs)
    
    for j, tableau_2 in enumerate(cliffords_mod_pauli[i+1:]):
        a2, b2, c2, d2, v2, w2 = tableau_2.to_numpy()
        assert (v1 == 0).all() and (w1 == 0).all()
        if (a1 == a2[:,::-1]).all() and (b1 == b2[:,::-1]).all() and (c1 == c2[:,::-1]).all() and (d1 == d2[:,::-1]).all():
            cliffords_mod_pauli_swappairs.append((tableau_1, tableau_2))
            swap_equiv_circuits_inds[i] = j+i+1 # the enumerate starts at i+1:
            swap_equiv_circuits_inds[j+i+1] = i # the enumerate starts at i+1:
            #print(i,j)
            break

print(len(cliffords_mod_pauli_swappairs))
print(len(set(swap_equiv_circuits_inds)))
print(len(set(swap_equiv_circuits_inds.values())))
assert {v:k for k,v in swap_equiv_circuits_inds.items()} == swap_equiv_circuits_inds

360
720
720


In [10]:
def swap_tableau(tableau):
    a, b, c, d, v1, v2 = tableau.to_numpy()
    a, b, c, d = a[:,::-1], b[:,::-1], c[:,::-1], d[:,::-1]
    v1, v2 = v1[::-1], v2[::-1]
    return stim.Tableau.from_numpy(x2x=a, x2z=b, z2x=c, z2z=d, x_signs=v1, z_signs=v2)

In [11]:
for i, j in swap_equiv_circuits_inds.items():
    assert cliffords_mod_pauli[i] == swap_tableau(cliffords_mod_pauli[j])

Now, we are going to optimize the circuits for each pair (C1, C1+SWAP) and choose the shallowest one. 

The optimization has several steps:
1. Optimize the CNOTs and converting them into SWAPs and propagating them till the end
2. Cancel out gates as `O^2 = 1 mod Paulis`

Let's now optimize the CNOT gates. 

In [12]:
def tableau_to_circuit(tableau):
    circuit = tableau.to_circuit(method="elimination")
    assert circuit.num_qubits == 2
    num_qubits = 2
    # split all gates in the circuit into pairs because can be e.g.
    # CX 0 2 0 1 or S 0 0
    # add a tick or the instructions are going to be compressed again
    circuit_tmp = stim.Circuit()
    for instr in circuit.flattened():
        if instr.name != "CX":
            for t in instr.targets_copy():
                circuit_tmp.append(stim.CircuitInstruction(instr.name, targets=[t]))
                circuit_tmp.append(stim.CircuitInstruction("TICK"))
            continue

        t = instr.targets_copy()
        pairs = zip(*[iter(t)] * 2)
        for pair in pairs:
            circuit_tmp.append(stim.CircuitInstruction("CX", targets=pair))
            circuit_tmp.append(stim.CircuitInstruction("TICK"))

    # split circuit into blocks that have only one operation per qubit
    blocks = []
    curr_block = []
    num_gates_qubits = np.zeros(num_qubits, dtype=int)
    for instr in circuit_tmp.flattened():
        if instr.name == "TICK":
            continue

        qubits = np.array([t.value for t in instr.targets_copy()])
        num_gates_qubits[qubits] += 1

        if (num_gates_qubits > 1).any():
            blocks.append(curr_block)
            curr_block = [instr]
            num_gates_qubits = np.zeros(num_qubits, dtype=int)
            num_gates_qubits[qubits] += 1
        else:
            curr_block.append(instr)
    blocks.append(curr_block)

    # merge blocks and separate them by TICKs
    circuit = stim.Circuit()
    circuit.append(stim.CircuitInstruction("R", targets=list(range(num_qubits))))
    circuit.append(stim.CircuitInstruction("TICK"))
    for block in blocks:
        for instr in block:
            circuit.append(instr)
        circuit.append(stim.CircuitInstruction("TICK"))
    circuit.append(stim.CircuitInstruction("M", targets=list(range(num_qubits))))

    return circuit

def ops_to_circuit(ops):
    circuit = stim.Circuit()
    for t, line in enumerate(ops):
        cnots = []
        for k, op in enumerate(line):
            if ("CX" in op):
                if (op not in cnots):
                    circuit += stim.Circuit(op.replace("CX", "CX ").replace("-", " "))
                    cnots.append(op)
            else:
                circuit += stim.Circuit(f"{op} {k}")
        if t != len(ops) - 1:
            circuit += stim.Circuit("TICK")
    return circuit

def remove_signs_tableau(tableau):
    a, b, c, d, v1, v2 = tableau.to_numpy()
    num_qubits = len(v1)
    return stim.Tableau.from_numpy(x2x=a, x2z=b, z2x=c, z2z=d, x_signs=np.zeros(num_qubits, dtype=bool), z_signs=np.zeros(num_qubits, dtype=bool))

In [13]:
def reverse_line(line):
    if ("CX0-1" in line) or ("CX1-0") in line:
        line = ["CX0-1", "CX0-1"] if "CX1-0" in line else ["CX1-0", "CX1-0"]
    line = line[::-1]
    return line

def improve_cnots_in_2q_ops(ops, initial_reverse):
    ops = ops.tolist()
    new_ops = []
    curr_cnot_block = []
    assert isinstance(initial_reverse, bool)
    reverse = initial_reverse
    for line in ops:
        if reverse:
            line = reverse_line(line)
            
        if (("CX0-1" in line) or ("CX1-0" in line)) and (len(curr_cnot_block) < 3):
            curr_cnot_block.append(line[0])
            continue

        if len(curr_cnot_block) == 0:
            new_ops.append(line)
            continue

        # process the CNOTs
        if len(curr_cnot_block) == 1:
            new_ops.append([curr_cnot_block[0], curr_cnot_block[0]])
        elif len(curr_cnot_block) > 3:
            print(curr_cnot_block)
            raise ValueError
        elif curr_cnot_block == ["CX0-1", "CX0-1"] or curr_cnot_block == ["CX1-0", "CX1-0"]:
            pass
        elif curr_cnot_block == ["CX1-0", "CX0-1"] or curr_cnot_block == ["CX0-1", "CX1-0"]:
            if curr_cnot_block == ["CX1-0", "CX0-1"]:
                new_ops.append(["CX1-0", "CX1-0"])
                reverse ^= True
                line = reverse_line(line)
            else:
                new_ops.append(["CX0-1", "CX0-1"])
                reverse ^= True
                line = reverse_line(line)
        elif curr_cnot_block == ["CX1-0", "CX0-1", "CX1-0"] or curr_cnot_block == ["CX0-1", "CX1-0", "CX0-1"]:
            reverse ^= True
            line = reverse_line(line)
            
        curr_cnot_block = []    

        new_ops.append(line)
            
    return new_ops, reverse

Let's see the difference when simplifying the CNOTs into swaps

In [14]:
circuit = tableau_to_circuit(cliffords_mod_pauli[712])
ops = get_ops(circuit)

In [15]:
ops.tolist()

[['R', 'R'],
 ['CX1-0', 'CX1-0'],
 ['CX0-1', 'CX0-1'],
 ['CX1-0', 'CX1-0'],
 ['H', 'S'],
 ['CX0-1', 'CX0-1'],
 ['I', 'H'],
 ['CX1-0', 'CX1-0'],
 ['I', 'S'],
 ['I', 'H'],
 ['M', 'M']]

In [16]:
improve_cnots_in_2q_ops(ops, initial_reverse=False)[0]

[['R', 'R'],
 ['S', 'H'],
 ['CX1-0', 'CX1-0'],
 ['H', 'I'],
 ['CX0-1', 'CX0-1'],
 ['S', 'I'],
 ['H', 'I'],
 ['M', 'M']]

Let's check that the `improve_cnots_in_2q_ops` does not change the effect of the circuits

In [17]:
# first check that everything does what it should do
for tableau in cliffords_mod_pauli:
    circuit = tableau_to_circuit(tableau)
    ops = get_ops(circuit)
    equiv_circuit = ops_to_circuit(ops)
    if circuit[1:-1].to_tableau() != remove_signs_tableau(equiv_circuit[1:-1].to_tableau()):
        print(circuit, "\n", equiv_circuit, "\n-------")

In [18]:
for tableau in cliffords_mod_pauli:
    circuit = tableau_to_circuit(tableau)
    ops = get_ops(circuit)
    ops, reverse = improve_cnots_in_2q_ops(ops, initial_reverse=False)
    equiv_circuit = ops_to_circuit(ops)
    # these two cases need to be considered separately because their tableaus are empty!!
    if circuit == stim.Circuit("R 0 1\nTICK\nCX 1 0\nTICK\nCX 0 1\nTICK\nCX 1 0\nTICK\nM 0 1"):
        if equiv_circuit == stim.Circuit("R 0 1\nTICK\nM 0 1"):
            continue
        else:
            print(circuit, "\n", equiv_circuit, "\n-------")
            break
    if circuit == stim.Circuit("R 0 1\nTICK\nH 1\nTICK\nH 1\nTICK\nM 0 1"):
        if equiv_circuit == stim.Circuit("R 0 1\nTICK\nI 0\nH 1\nTICK\nI 0\nH 1\nTICK\nM 0 1"):
            continue
        else:
            print(circuit, "\n", equiv_circuit, "\n-------")
            break
    else:
        if reverse:
            if remove_signs_tableau(circuit[1:-1].to_tableau()) != remove_signs_tableau(swap_tableau(equiv_circuit[1:-1].to_tableau())):
                print(circuit, "\n", equiv_circuit, "\n-------")
                break
        else:
            if remove_signs_tableau(circuit[1:-1].to_tableau()) != remove_signs_tableau(equiv_circuit[1:-1].to_tableau()):
                print(circuit, "\n", equiv_circuit, "\n-------")
                break

Now we also cancel out the gates and avoid unnecessary idling times.

In [21]:
from itertools import zip_longest

def grouper(iterable, n, fillvalue=None):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def ops_to_compact_circuit(ops):
    circuit = stim.Circuit()
    for t, line in enumerate(ops):
        cnots = []
        for k, op in enumerate(line):
            if ("CX" in op):
                if (op not in cnots):
                    circuit += stim.Circuit(op.replace("CX", "CX ").replace("-", " "))
                    cnots.append(op)
            elif "I" == op:
                continue
            else:
                circuit += stim.Circuit(f"{op} {k}")
    return circuit

def compact_circuit_to_circuit(ccircuit): 
    assert ccircuit.num_qubits == 2
    
    circuit = stim.Circuit()
    curr_gates = [[], []]

    def process_gates():
        circuit = stim.Circuit()
        for g1, g2 in zip_longest(curr_gates[0], curr_gates[1], fillvalue="I"):
            circuit += stim.Circuit(f"{g1} 0\n{g2} 1\nTICK")
        return circuit
        
    for instr in ccircuit.flattened():
        if instr.name == "CX":
            circuit += process_gates()
            # avoid CX 0 1 1 0 0 1, should be CX 0 1 TICK 1 0 TICK 0 1
            for ts in grouper(instr.targets_copy(), 2):
                new_instr = stim.CircuitInstruction("CX", targets=ts)
                circuit.append(new_instr) # add CX
                circuit += stim.Circuit("TICK")
            curr_gates = [[], []]
            continue

        if instr.name == "M":
            circuit += process_gates()
            circuit.append(instr)
            curr_gates = [[], []]
            continue

        for target in instr.targets_copy():
            t = target.value
            if (len(curr_gates[t]) != 0) and (curr_gates[t][-1] == instr.name):
                # repeated gate
                curr_gates[t] = curr_gates[t][:-1]
            else:
                curr_gates[t].append(instr.name)
            
    if curr_gates != [[], []]:
        raise ValueError # the circuit should end in M 0 1
                
    return circuit

In [22]:
ops = (('R', 'R'), ('I', 'S'), ('S', 'H'), ('H', 'I'), ('S', 'H'), ('I', 'H'), ('M', 'M'))
circuit_long = ops_to_circuit(ops)
ccircuit = ops_to_compact_circuit(ops)
circuit = compact_circuit_to_circuit(ccircuit)
print(circuit_long)
print("-----")
print(circuit)

R 0 1
TICK
I 0
S 1
TICK
S 0
H 1
TICK
H 0
I 1
TICK
S 0
H 1
TICK
I 0
H 1
TICK
M 0 1
-----
R 0 1
TICK
S 0 1
TICK
H 0 1
TICK
S 0
I 1
TICK
M 0 1


Let's check that the functions do not change the circuits:

In [23]:
for tableau in cliffords_mod_pauli:
    circuit_long = tableau_to_circuit(tableau)
    ops = get_ops(circuit_long)
    ccircuit = ops_to_compact_circuit(ops)
    circuit = compact_circuit_to_circuit(ccircuit)

    if circuit_long == stim.Circuit("R 0 1\nTICK\nCX 1 0\nTICK\nCX 0 1\nTICK\nCX 1 0\nTICK\nM 0 1"):
        if circuit == stim.Circuit("R 0 1\nTICK\nCX 1 0\nTICK\nCX 0 1\nTICK\nCX 1 0\nTICK\nM 0 1"):
            continue
        else:
            print(circuit_long, "\n", circuit, "\n-------")
            break
    if circuit_long == stim.Circuit("R 0 1\nTICK\nH 1\nTICK\nH 1\nTICK\nM 0 1"):
        if circuit == stim.Circuit("R 0 1\nTICK\nM 0 1"):
            continue
        else:
            print(circuit_long, "\n", circuit, "\n-------")
            break
    
    if remove_signs_tableau(circuit_long[1:-1].to_tableau()) != remove_signs_tableau(circuit[1:-1].to_tableau()):
        print(circuit_long, "--", circuit)
        break

Now that we have all the ingredients, we create a function that does everything together. And then we also check that it does what it is supposed to do. 

In [24]:
def simplify_circuit(circuit, initial_reverse):
    ops = get_ops(circuit)
    ops, reverse = improve_cnots_in_2q_ops(ops, initial_reverse=initial_reverse)
    circuit_long = ops_to_circuit(ops)
    ccircuit = ops_to_compact_circuit(ops)
    circuit = compact_circuit_to_circuit(ccircuit)
    return circuit, reverse

In [25]:
for tableau in cliffords_mod_pauli:
    circuit = tableau_to_circuit(tableau)
    equiv_circuit, reverse = simplify_circuit(circuit, initial_reverse=False)
    
    # these two cases need to be considered separately because their tableaus are empty!!
    if circuit == stim.Circuit("R 0 1\nTICK\nCX 1 0\nTICK\nCX 0 1\nTICK\nCX 1 0\nTICK\nM 0 1"):
        if equiv_circuit == stim.Circuit("R 0 1\nTICK\nM 0 1"):
            continue
        else:
            print(circuit, "\n", equiv_circuit, "\n-------")
            break
    if circuit == stim.Circuit("R 0 1\nTICK\nH 1\nTICK\nH 1\nTICK\nM 0 1"):
        if equiv_circuit == stim.Circuit("R 0 1\nTICK\nM 0 1"):
            continue
        else:
            print(circuit, "\n", equiv_circuit, "\n-------")
            break
    else:
        if reverse:
            if remove_signs_tableau(circuit[1:-1].to_tableau()) != remove_signs_tableau(swap_tableau(equiv_circuit[1:-1].to_tableau())):
                print(circuit, "\n", equiv_circuit, "\n-------")
                break
        else:
            if remove_signs_tableau(circuit[1:-1].to_tableau()) != remove_signs_tableau(equiv_circuit[1:-1].to_tableau()):
                print(circuit, "\n", equiv_circuit, "\n-------")
                break

Perfect, so it works. Let's benchmark its performance in terms of circuit depth. 

In [26]:
def get_depth_from_circuit(circuit):
    circuit_str = str(circuit)
    depth = len(circuit_str.split("TICK"))
    return depth

In [27]:
depths = []
initial_depths = []
CIRCUITS = []
REVERSES = []

for tableau in cliffords_mod_pauli:
    circuit = tableau_to_circuit(tableau)
    equiv_circuit, reverse = simplify_circuit(circuit, initial_reverse=False)
    depth = get_depth_from_circuit(equiv_circuit)

    depths.append(depth - 2) # for M and R
    CIRCUITS.append(equiv_circuit)
    REVERSES.append(reverse)
    initial_depths.append(get_depth_from_circuit(circuit))

In [28]:
# check that the circuit tableau and the original tableau are the same
for i, tableau in enumerate(cliffords_mod_pauli):
    circuit_tableau = (CIRCUITS[i][1:-1] + stim.Circuit("I 0 1")).to_tableau()
    if REVERSES[i]:
        assert remove_signs_tableau(swap_tableau(circuit_tableau)) == tableau
    else:
        assert remove_signs_tableau(circuit_tableau) == tableau

In [29]:
np.average(initial_depths), np.max(initial_depths)

(10.0625, 20)

In [30]:
np.average(depths), np.max(depths)

(4.552777777777778, 9)

In [31]:
len([i for i in REVERSES if i])

360

In [32]:
print(len(set([tuple(tuple(i) for i in get_ops(c)) for c in CIRCUITS])), len(CIRCUITS))

666 720


Now we are going to choose the shallowest circuit from the "SWAP pair"

In [33]:
CIRCUITS_MOD_SWAP = []
REVERSES_MOD_SWAP = []
processed = []
depths = []

for i, circuit in enumerate(CIRCUITS):
    if i in processed:
        continue
        
    circuit_swap = CIRCUITS[swap_equiv_circuits_inds[i]]
    processed.append(swap_equiv_circuits_inds[i])
    
    # ensure that one of the two circuits ends with a SWAP
    assert REVERSES[i] ^ REVERSES[swap_equiv_circuits_inds[i]] == True

    if get_depth_from_circuit(circuit) <= get_depth_from_circuit(circuit_swap):
        CIRCUITS_MOD_SWAP.append(circuit)
        REVERSES_MOD_SWAP.append(REVERSES[i])
    else:
        CIRCUITS_MOD_SWAP.append(circuit_swap)
        REVERSES_MOD_SWAP.append(REVERSES[swap_equiv_circuits_inds[i]])

In [34]:
print(len(CIRCUITS_MOD_SWAP), len(REVERSES_MOD_SWAP))

360 360


In [35]:
depths = []
for circuit in CIRCUITS_MOD_SWAP:
    depth = get_depth_from_circuit(circuit) - 2 # M and R
    depths.append(depth)
print(len(depths), np.average(depths), np.max(depths))

360 4.052777777777778 8


In [36]:
depths = []
for circuit in CIRCUITS:
    depth = get_depth_from_circuit(circuit) - 2 # M and R
    depths.append(depth)
print(len(depths), np.average(depths), np.max(depths))

720 4.552777777777778 9


In [37]:
# unique circuits
len(set([tuple(tuple(l) for l in get_ops(c)) for c in CIRCUITS_MOD_SWAP]))

360

Finally, we are going to find the inverse of each circuit (it should be present in the group). 

In [38]:
def get_tableau_from_circuit(circuit):
    return (circuit[1:-1] + stim.Circuit("I 0 1")).to_tableau()

def swap_circuit(circuit):
    ops = tuple(tuple(l) for l in get_ops(circuit))
    swap_ops = [l[::-1] for l in ops]
    swap_circuit = ops_to_circuit(swap_ops)
    return swap_circuit

In [39]:
CIRCUITS_INV_MOD_SWAP = []
for i, (circuit, _) in enumerate(zip(CIRCUITS_MOD_SWAP, REVERSES_MOD_SWAP)):
    print(i, "\r", end="")
    prev_len = len(CIRCUITS_INV_MOD_SWAP)
    t1 = remove_signs_tableau(get_tableau_from_circuit(circuit))
    
    for other_circuit, _ in zip(CIRCUITS_MOD_SWAP, REVERSES_MOD_SWAP):
        t2 = remove_signs_tableau(get_tableau_from_circuit(other_circuit))
        t1_inv = t1.inverse(unsigned=True) # all signs are +1
        if t1_inv == t2:
            CIRCUITS_INV_MOD_SWAP.append(other_circuit)
            break
        t2 = swap_tableau(remove_signs_tableau(get_tableau_from_circuit(other_circuit)))
        t1_inv = t1.inverse(unsigned=True) # all signs are +1
        if t1_inv == t2:
            other_circuit = swap_circuit(other_circuit)
            CIRCUITS_INV_MOD_SWAP.append(other_circuit)
            break

    if len(CIRCUITS_INV_MOD_SWAP) != prev_len + 1:
        print(i)
        break

In [40]:
print(len(CIRCUITS_INV_MOD_SWAP))

360


In [42]:
# count how many times the inverse circuit and the circuit are the same but reversed
num_equivalent = 0
for circuit, circuit_inv in zip(CIRCUITS_MOD_SWAP, CIRCUITS_INV_MOD_SWAP):
    ops = tuple(tuple(l) for l in get_ops(circuit))
    ops_inv = tuple(tuple(l) for l in get_ops(circuit_inv))
    if ops[1:-1] == ops_inv[1:-1][::-1]:
        num_equivalent += 1
    
print(num_equivalent, len(CIRCUITS_MOD_SWAP), num_equivalent/len(CIRCUITS_MOD_SWAP))

83 360 0.23055555555555557


In [43]:
83/360

0.23055555555555557

In [44]:
equivalent = []
for circuit, circuit_inv in zip(CIRCUITS_MOD_SWAP, CIRCUITS_INV_MOD_SWAP):
    ops = tuple(tuple(l) for l in get_ops(circuit))
    ops_inv = tuple(tuple(l) for l in get_ops(circuit_inv))
    if ops[1:-1] == ops_inv[1:-1][::-1]:
        equivalent.append(circuit)

In [45]:
len(equivalent)

83

In [49]:
sq_equivalent = []
tq_equivalent = []
for c in equivalent:
    if "CX" in str(c):
        tq_equivalent.append(c)
    else:
        sq_equivalent.append(c)

print(len(sq_equivalent))
print(len(tq_equivalent))

20
63


In [54]:
short_equivalent = []
long_equivalent = []
for c in tq_equivalent:
    if str(c).count("TICK") < 5:
        short_equivalent.append(c)
    else:
        long_equivalent.append(c)

print(len(sq_equivalent))
print(len(short_equivalent))
print(len(long_equivalent))

20
40
23


In [62]:
short_equivalent[5]

stim.Circuit('''
    R 0 1
    TICK
    CX 1 0
    TICK
    I 0
    H 1
    TICK
    I 0
    S 1
    TICK
    M 0 1
''')

In [58]:
long_equivalent[1]

stim.Circuit('''
    R 0 1
    TICK
    CX 0 1
    TICK
    S 0
    I 1
    TICK
    H 0
    I 1
    TICK
    S 0
    I 1
    TICK
    M 0 1
''')

Let's now do the final check where we check that:
1. all the appropiate tableaus are in `CIRCUITS_MOD_SWAP`
2. all the circuits in `CIRCUITS_INV_MOD_SWAP` are actually the inverse circuits of each corresponding `CIRCUITS_MOD_SWAP`

In [38]:
assert len(CIRCUITS_MOD_SWAP) == len(CIRCUITS) // 2

for circuit in CIRCUITS_MOD_SWAP:
    tableau = remove_signs_tableau(get_tableau_from_circuit(circuit))

    found = False
    for t1, t2 in cliffords_mod_pauli_swappairs:
        if t1 == tableau:
            found = True
            break
        if t2 == tableau:
            found = True
            break

    assert found

In [39]:
for circuit, circuit_inv in zip(CIRCUITS_MOD_SWAP, CIRCUITS_INV_MOD_SWAP):
    total_circuit = circuit[1:-1] + circuit_inv[1:-1] + stim.Circuit("I 0 1")
    tableau = remove_signs_tableau(total_circuit.to_tableau())
    assert tableau == stim.Tableau(2)

In [40]:
depths = []
for circuit, circuit_inv in zip(CIRCUITS_MOD_SWAP, CIRCUITS_INV_MOD_SWAP):
    total_circuit = circuit[:-1] + circuit_inv[2:] # remove M and R+TICK
    depth = get_depth_from_circuit(total_circuit) - 2 # M and R
    depths.append(depth)
print(len(depths), np.average(depths), np.max(depths))

360 8.105555555555556 14


Now, let's store all these circuits into a file.

In [41]:
data = ""
for circuit, circuit_inv in zip(CIRCUITS_MOD_SWAP, CIRCUITS_INV_MOD_SWAP):
    total_circuit = circuit[:-1] + circuit_inv[2:] # remove M and R+TICK
    data += f"CIRCUIT:\n{circuit}\nCIRCUIT INVERSE:\n{circuit_inv}\nTOTAL CIRCUIT:\n{total_circuit}\n----------\n"

print(data)

CIRCUIT:
R 0 1
TICK
M 0 1
CIRCUIT INVERSE:
R 0 1
TICK
M 0 1
TOTAL CIRCUIT:
R 0 1
TICK
M 0 1
----------
CIRCUIT:
R 0 1
TICK
I 0
S 1
TICK
I 0
H 1
TICK
I 0
S 1
TICK
M 0 1
CIRCUIT INVERSE:
R 0 1
TICK
I 0
S 1
TICK
I 0
H 1
TICK
I 0
S 1
TICK
M 0 1
TOTAL CIRCUIT:
R 0 1
TICK
I 0
S 1
TICK
I 0
H 1
TICK
I 0
S 1
TICK
I 0
S 1
TICK
I 0
H 1
TICK
I 0
S 1
TICK
M 0 1
----------
CIRCUIT:
R 0 1
TICK
I 0
H 1
TICK
M 0 1
CIRCUIT INVERSE:
R 0 1
TICK
I 0
H 1
TICK
M 0 1
TOTAL CIRCUIT:
R 0 1
TICK
I 0
H 1
TICK
I 0
H 1
TICK
M 0 1
----------
CIRCUIT:
R 0 1
TICK
I 0
H 1
TICK
I 0
S 1
TICK
M 0 1
CIRCUIT INVERSE:
R 0 1
TICK
I 0
S 1
TICK
I 0
H 1
TICK
M 0 1
TOTAL CIRCUIT:
R 0 1
TICK
I 0
H 1
TICK
I 0
S 1
TICK
I 0
S 1
TICK
I 0
H 1
TICK
M 0 1
----------
CIRCUIT:
R 0 1
TICK
I 0
S 1
TICK
I 0
H 1
TICK
M 0 1
CIRCUIT INVERSE:
R 0 1
TICK
I 0
H 1
TICK
I 0
S 1
TICK
M 0 1
TOTAL CIRCUIT:
R 0 1
TICK
I 0
S 1
TICK
I 0
H 1
TICK
I 0
H 1
TICK
I 0
S 1
TICK
M 0 1
----------
CIRCUIT:
R 0 1
TICK
I 0
S 1
TICK
M 0 1
CIRCUIT INVERSE:
R 0 1
TICK
I 

In [43]:
with open("20250130_cliffords_modulo_paulis_and_swap.txt", "w") as file:
    file.write(data)

In [69]:
circuits = [block.split("TOTAL CIRCUIT:\n")[1] for block in data.split("\n----------\n") if block != ""]
labelled_circuits_z = {}
labelled_circuits_x = {}
for k, circuit in enumerate(circuits):
    labelled_circuits_z[k] = circuit
    labelled_circuits_x[k] = circuit.replace("R 0 1", "RX 0 1").replace("M 0 1", "MX 0 1")
    
EXPERIMENTS = list(range(len(labelled_circuits_z)))
print(len(EXPERIMENTS))

360
